# Macroprudential Policy and the Housing Market
This notebook solves and simulates the Heterogenous Agent Housing Market (HAH) model and produces the output contained in my Master's thesis. 

**Dependencies:**

The code structure builds upon the framework developed by Jeppe Druedahl & Co. in the [NumEconCopenhagen Project](https://github.com/NumEconCopenhagen)

Packages required for running the notebooks are:
- [ConSav](https://pypi.org/project/ConSav/)
- [EconModel](https://pypi.org/project/EconModel/)
- [matplotlib](https://pypi.org/project/matplotlib/)
- [numpy](https://pypi.org/project/numpy/)
- [numba](https://pypi.org/project/numba/)

In [4]:
# imports
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import numba as nb
nb.set_num_threads(4)

from HAHModel import HAHModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Baseline model

In [10]:
# initialise an instance of baseline model
model_baseline = HAHModelClass(
    name='baseline',
    par={
        'do_print':True
        })

# solve model with very coarse grids --> speed gains
model_baseline.precompile_numba() 

numba precompiled in 86.9 secs


In [11]:
model_baseline.solve(do_assert=False) 
#model_baseline.simulate()

 last period bequest computed computed in 0.0 secs
 solved stayer problem in 0.2 secs
 solved refinance problem in 4.0 secs
 solved buyer problem in 26.2 secs
 solved renter problem in 0.0 secs
 t = 54 solved in 30.4 secs
 v_bar and q computed in 8.9 secs
 solved stayer problem in 0.1 secs
 solved refinance problem in 3.7 secs
 solved buyer problem in 26.2 secs
 solved renter problem in 0.0 secs
 t = 53 solved in 69.3 secs
 v_bar and q computed in 8.7 secs


ZeroDivisionError: division by zero

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/Christian/Dropbox/KU - Polit/KA/Thesis/MScThesis-2022/HAHModel.py", line 383, in solve
    hhp.solve_stay(t,sol,par)
SystemError: CPUDispatcher(<function solve_stay at 0x12f9474c0>) returned a result with an error set


SystemError: CPUDispatcher(<function solve_stay at 0x12f9474c0>) returned a result with an error set

### Test solving the household problem for a single period

In [15]:
import trans
import HHproblems as hhp
from EconModel import jit

par = model_baseline.par

t = par.T-1

with jit(model_baseline) as model: 
    sol = model.sol
    par = model.par
    hhp.last_period_v_bar_q(t,sol,par)
    print('bequest is done')
    hhp.solve_stay(t,sol,par)
    print('stay is done')
    hhp.solve_ref(t,sol,par)
    print('refinance is done')
    hhp.solve_buy(t,sol,par)
    print('buy is done')
    hhp.solve_rent(t,sol,par)
    print('rent is done')
    t = par.T-2
    hhp.postdecision_compute_v_bar_q(t,sol,par)
    print('post decision is done')
    #assert np.all((sol.c_stay[t] >= 0) & (np.isnan(sol.c_stay[t]) == False))
    #assert np.all((sol.inv_v_bar[t] >= 0) & (np.isnan(sol.inv_v_bar[t]) == False))


bequest is done
stay is done
refinance is done
buy is done
rent is done
Iteration number 100
Iteration number 100
Iteration number 100
Iteration number 100
Iteration number 200
Iteration number 200
Iteration number 200
Iteration number 200
Iteration number 300
Iteration number 300
Iteration number 300
Iteration number 300
Iteration number 400
Iteration number 400
Iteration number 400
Iteration number 400
Iteration number 500
Iteration number 500
Iteration number 500
Iteration number 500
Iteration number 600
Iteration number 600
Iteration number 600
Iteration number 700
Iteration number 600
Iteration number 700
Iteration number 700
Iteration number 800
Iteration number 700
Iteration number 800
Iteration number 800
Iteration number 900
Iteration number 800
Iteration number 900
Iteration number 1000
Iteration number 900
Iteration number 900
Iteration number 1000
Iteration number 1100
Iteration number 1000
Iteration number 1100
Iteration number 1000
Iteration number 1200
Iteration number 1

In [5]:
sol = model_baseline.sol 
par = model_baseline.par
t = par.T-2

print(sol.c_buy.shape)
print(sol.c_stay.shape)

#for i_ht in range(par.Nhtilde):
#    print(sol.inv_v_rent[t+1,:,i_ht,:])

(55, 7, 10, 24, 11, 7, 10)
(55, 6, 10, 24, 11, 7, 10)


### Testing utility functions, tax scheme and mortgage payment

In [16]:
import mt
import utility

with jit(model_baseline) as model_jit:
    par = model_jit.par
    
    # mortgage schedule
    annuity,pr_rem,interest,pr_pmt = mt.mpmt(10**6,1,20,0,par)

    # income and property tax
    tax_h = mt.property_tax(1.0,2.0,par)
    ytilde = mt.income_aftertax(2.2,0.8,3.5,0,par)

    # utility
    u_stay = utility.func(1,2,0,0,5,par)
    marg_u_stay = utility.marg_func(2,par,par.T-1)
    u_move = utility.func(1,2,1,0,5,par)
    u_rent = utility.func(1,2,1,1,5,par)

# print output
print(f'the full annuity payment is {round(annuity,2)}')
print(f'the remaining principal is {round(pr_rem,2)}')
print(f'the interest payment is {round(interest,2)}')
print(f'the principal payment is {round(pr_pmt,2)}')
print('--------------------------------------------')
print(f'the property tax is {round(tax_h,2)}')
print(f'the after tax income is {round(ytilde,2)}')
print('--------------------------------------------')
print(f'utility for a stayer is {round(u_stay,2)}')
print(f'marginal stay utility is {round(marg_u_stay,2)}')
print(f'utility for a mover is {round(u_move,2)}')
print(f'utility for a renter is {round(u_rent,2)}')


the full annuity payment is 67215.71
the remaining principal is 962784.29
the interest payment is 30000.0
the principal payment is 37215.71
--------------------------------------------
the property tax is 0.02
the after tax income is 1.57
--------------------------------------------
utility for a stayer is -3.17
marginal stay utility is 0.18
utility for a mover is -3.51
utility for a renter is -3.55


### Testing transition rules

In [34]:
# da periods
Tda_next = trans.Tda_plus_func(10)
print(Tda_next)

m_plus_net_stay = np.zeros(5)
m_plus = np.ones(5)

with jit(model_baseline) as model:
    par = model.par
   
    # mortgage balance
    d_plus = trans.d_plus_func(q=1,h=2,d=1.5,w=0.8,t=5,Td=20,Tda=0,par=par)
    for i in range(5):
        m_plus_net_stay[i] = trans.m_to_mnet_stay(m_plus=m_plus[i],h=4,par=par)

print(f'next period mortgage balance is {d_plus}') 
print(f'm_plus_net_stay is {m_plus_net_stay}')

# income states

9
next period mortgage balance is 1.4441764386047113
m_plus_net_stay is [0.9032 0.9032 0.9032 0.9032 0.9032]


### Sanity checks

In [41]:
sol = model_baseline.sol
par = model_baseline.par
t = par.T-2
for i_ht in range(3):
    print(sol.inv_v_rent[t+1,:,i_ht,2])
    print(sol.inv_v_rent[t+1,:,i_ht,:].ndim)


[0.         0.04671375 0.05580381 0.05945007 0.06213019 0.0660677
 0.0718353  0.08015892 0.09191713 0.10803498]
2
[0.         0.04687412 0.05603282 0.05971006 0.06241427 0.06638901
 0.0722153  0.08063236 0.09254017 0.10889668]
2
[0.         0.04703268 0.05625954 0.05996758 0.06269579 0.06670761
 0.07259242 0.08110277 0.09316028 0.10975635]
2


In [17]:
model_baseline.par.w_grid 
model_baseline.par.w_trans.round(2)

array([[0.89, 0.11, 0.01, 0.  , 0.  , 0.  , 0.  ],
       [0.02, 0.89, 0.09, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.04, 0.89, 0.07, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.05, 0.89, 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.07, 0.89, 0.04, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.09, 0.89, 0.02],
       [0.  , 0.  , 0.  , 0.  , 0.01, 0.11, 0.89]])